In [ ]:
import pandas as pd

In [ ]:
!nvidia-smi

Fri Apr  8 20:18:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install pickle
import pickle as pkl
df= pkl.load(open('df.pkl','rb'))
new_df= pd.DataFrame(df)

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


FileNotFoundError: ignored

In [ ]:
new_df.columns

In [ ]:
# submitter,	authors, title, comments, categories, abstract

In [ ]:
f_df = new_df[['id','submitter','authors','title','categories','abstract','update_date']]

In [ ]:
f_df=f_df.head(20000)

In [ ]:
f_df.shape

In [ ]:
f_df.drop(['submitter'],axis=1,inplace=True)

In [ ]:
idx = [i for i in  range(0,50000)]

In [ ]:
idx = pd.DataFrame(idx)

In [ ]:
f_df.head()

In [ ]:
import string
def helper(x):
  temp = str.maketrans('','',string.punctuation)
  return x.translate(temp)

In [ ]:
f_df['authors'].values

In [ ]:
f_df['authors'] = f_df['authors'].apply(helper)

In [ ]:
f_df.head()

In [ ]:
f_df['title'] = f_df['title'].apply(helper)

In [ ]:
f_df.head(10)

In [ ]:
st = 'math.CO cs.CG'
st.split('.')

In [ ]:
f_df['categories'] = f_df['categories'].apply(lambda x:x.split())

In [ ]:
f_df.head()

In [ ]:
f_df['abstract'] = f_df['abstract'].apply(helper)

In [ ]:
f_df.iloc[30:40]

In [ ]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words



def rem_stopwords(sent):
    cleaned = []
    for word in sent.split():
        if word.lower() not in stopwords:
          cleaned.append(word)
    return " ".join(cleaned)


In [ ]:
f_df['abstract'] =  f_df['abstract'].apply(rem_stopwords)

In [ ]:

f_df['date']= f_df['update_date']
f_df.drop(columns=['update_date'],inplace=True)
f_df.head()


In [ ]:
f_df['authors'] = f_df['authors'].apply(lambda x:x.split('and'))

In [ ]:
f_df.head()

In [ ]:
mylist = ['Ileana Streinu' , 'Louis Theran']
[elem for elem in mylist if elem.strip()]

In [ ]:
def rem_space(lis):
  final_lis = []
  for i in lis:
    i = i.replace(" ","")
    final_lis.append(i)
  return final_lis

In [ ]:
 f_df['authors'] = f_df['authors'].apply(rem_space)

In [ ]:
f_df.head()

In [ ]:
def tolower(x):
  lis = []
  for i in x:
    lis.append(i.lower())
  return lis

In [ ]:
f_df['authors'] = f_df['authors'].apply(tolower)

In [ ]:
f_df['abstract'] = f_df['abstract'].apply(lambda x:x.split())

In [ ]:
f_df['tags'] = f_df['authors'] + f_df['categories'] + f_df['abstract']

In [ ]:
f_df.drop(columns=['date'],inplace=True)
f_df.head()

In [ ]:
new_df = f_df[['id','title','tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x)) 

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
  y = []
  for word in text.split():
    y.append(ps.stem(word))
  return " ".join(y)


In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
# new_df.head()
new_df.shape

In [ ]:
new_df.shape[0]

In [ ]:
idx = [i for i in range(new_df.shape[0])]

In [ ]:
new_df.reset_index(inplace=True)

In [ ]:


x_df = pd.DataFrame(new_df,set(idx))


In [ ]:
x_df.drop('index',axis=1,inplace=True)

In [ ]:
x_df.shape

In [ ]:
def helper2(x):
    i = x.replace('\n','')
    i = i.replace('\r','')
    return i

In [ ]:
x_df['title'] = x_df['title'].apply(helper2)

In [ ]:
x_df['title'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
import numpy as np
vectors = cv.fit_transform(x_df['tags']).toarray()

In [ ]:
vectors.shape

In [ ]:
len(cv.get_feature_names())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [ ]:
similarity[1]

In [ ]:
x_df['title']  =  x_df['title'].apply(helper)

In [ ]:
x_df['title'][0]

In [ ]:
x_df[x_df['title'] =='Calculation of prompt diphoton production cross sections at Tevatron and  LHC energies' ].index

In [ ]:
def recommend(paper):
    lis = []
    paper_idx = x_df[x_df['title'] == paper].index[0]
    distance = similarity[paper_idx]
    paper_list = sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:10]
    for i in paper_list:
        id = x_df.iloc[i[0]].title
        lis.append(id)
    return lis

In [ ]:
# df[df['up'] == 0]
recommend('Mapping radii of metric spaces')


In [ ]:
f_df.categories.value_counts()

In [ ]:
# https://arxiv.org/pdf/{id}.pdf

In [ ]:
import pickle
pickle.dump(similarity,open('similarity1.pkl','wb'))

In [ ]:
pickle.dump(x_df.to_dict(),open('df1.pkl','wb'))